In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as data_set
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np


print(torch.__version__, torchvision.__version__)
print(torch.cuda.get_device_name(0))

1.8.0 0.2.2
TITAN RTX


In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

max_epochs = 30

In [3]:
vgg16 = torchvision.models.vgg16(pretrained=True).to(device)
#summary(vgg16, input_size=(224,224,3))

In [4]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5, 0.5, 0.5)),
])

In [5]:
'''
!ls /media/2/Network/Imagenet_dup
'''
dataset_path = "/media/2/Network/Imagenet_dup/"
# imagenet data load
train_dataset = data_set.ImageFolder(root=dataset_path+"train",
                                       transform=transform)
val_dataset = data_set.ImageFolder(root=dataset_path+"val",
                                       transform=transform)
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                        batch_size=32,
                                        shuffle=False,
                                        num_workers=0)
val_dataloader = torch.utils.data.DataLoader(val_dataset,
                                        batch_size=32,
                                        shuffle=False,
                                        num_workers=0)

In [6]:
len(train_dataloader),len(val_dataloader)

(40037, 1563)

In [7]:
# training phase
criterion = torch.nn.CrossEntropyLoss(size_average = None).to(device)
optimizer = torch.optim.SGD(vgg16.parameters(),lr=0.0005)
lr_sche = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)
'''
vgg16.to(device)
for epoch in range(max_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader):
        inputs,labels = data
        inputs,labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        y_pred = vgg16(inputs)
        # compute loss 
        loss = criterion(y_pred,labels)
        #print(epoch,i, loss.item())
        
        if not torch.isfinite(loss):
            print("WARNING: non-finite loss, ending training")
            exit(1)
        else :
            running_loss += loss.item()
        loss.backward()
        optimizer.step()
        if i % 30 == 29: 
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/30)) 
            running_loss = 0.0
            

'''


        
        

'\nvgg16.to(device)\nfor epoch in range(max_epochs):\n    running_loss = 0.0\n    for i, data in enumerate(train_dataloader):\n        inputs,labels = data\n        inputs,labels = inputs.to(device), labels.to(device)\n        optimizer.zero_grad()\n        \n        y_pred = vgg16(inputs)\n        # compute loss \n        loss = criterion(y_pred,labels)\n        #print(epoch,i, loss.item())\n        \n        if not torch.isfinite(loss):\n            print("WARNING: non-finite loss, ending training")\n            exit(1)\n        else :\n            running_loss += loss.item()\n        loss.backward()\n        optimizer.step()\n        if i % 30 == 29: \n            print(\'[%d, %5d] loss: %.3f\' % (epoch+1, i+1, running_loss/30)) \n            running_loss = 0.0\n            \n\n'

In [8]:
optimizer.param_groups[0]['lr']
    

0.0005

In [9]:
# evaluation pahse
val_losses=[]
vgg16.eval()
total = 0
correct =0
with torch.no_grad():
    for i, data in enumerate(train_dataloader):
        inputs,labels = data
        inputs,labels = inputs.to(device), labels.to(device)
        
        y_hat = vgg16(inputs)
        _, predicted = torch.max(y_hat.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(y_hat.size(),labels.size())
        val_loss = criterion(y_hat, labels)
        val_losses.append(val_loss.item())
        if(i%100 == 99):
            print(".",end="")
    print("")
print("acc of vgg16 on imagenet : %.4f %%" %(100*correct/total))    

torch.Size([32, 1000]) torch.Size([32])
torch.Size([32, 1000]) torch.Size([32])
torch.Size([32, 1000]) torch.Size([32])
torch.Size([32, 1000]) torch.Size([32])
torch.Size([32, 1000]) torch.Size([32])
torch.Size([32, 1000]) torch.Size([32])
torch.Size([32, 1000]) torch.Size([32])


KeyboardInterrupt: 

In [ ]:
print("acc of vgg16 on imagenet : %d %%" %(100*correct/total))    